# Image Data Augmentation

Please also refer to [Keras tutorial on image data augmentation](https://www.tensorflow.org/tutorials/images/data_augmentation)

# Setup

## Some config

In [ ]:
#an image of a chameleon
IMAGE_URL = 'https://raw.githubusercontent.com/ne1s0n/coding_excercises/master/data/800px-Panther_chameleon_(Furcifer_pardalis)_male_Nosy_Be.jpg'

#where to save it, locally
IMG_DIR = '/content'
IMG_FILENAME = IMG_DIR + '/cham.jpg'

#where to create the augmented data
DIR_AUGMENTED_DATA = '/content/augmented/'

## Support function

The next code block defines a function that:

* receives in input a `Sequential` model and the filepath to an image
* creates a 3x3 plot. In the first subplot the original image is plotted
* for each other subplot, the original image is passed through the model first. Since no random seed is managed the resulting eigth images should differ

In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt

def generate_plot_pics(augmentation_layers, image_path):
  '''
  Loads the image from the passed path, passes it the passed
  augmentation layers eigth different times (thus bearing eigth
  different results). Shows everything in a figure with the
  following subplot schema:
  +-----------+-----------+-----------+
  |     1     |     2     |     3     |
  |  original |   augm    |   augm    |
  +-----------+-----------+-----------+
  |     4     |     5     |     6     |
  |   augm    |   augm    |   augm    |
  +-----------+-----------+-----------+
  |     7     |     8     |     9     |
  |   augm    |   augm    |   augm    |
  +-----------+-----------+-----------+
  '''

  #load the image, convert it to a numpy array with the dimensions expected
  #by a tf model (we add batch number as the first dim)
  image = tf.keras.utils.load_img(image_path)
  image_array = tf.keras.utils.img_to_array(image)
  image_array = tf.expand_dims(image_array, axis=0)  # add batch dimension (1, h, w, 3)

  #prepare a 3x3 plot, with the original picture in position one
  fig = plt.figure(figsize=(8, 6))
  fig.subplots_adjust(hspace=0.02,wspace=0.01,
                  left=0,right=1,bottom=0, top=1)
  ax = fig.add_subplot(3, 3, 1,xticks=[],yticks=[])

  #the original image is currently in [0-255] range, but we
  #need to bring it to [0-1] so that pyplot can show it
  original_showable = image_array[0] / 255.0

  #adding it to the current figure
  ax.imshow(original_showable)
  ax.set_title("Original")

  #for each of the other positions, generate and add an augmented image
  for i in range(2,10):
    #generate the augmented image
    augmented_image = augmentation_layers(image_array)

    #convert tensors to numpy arrays for visualization
    augmented_showable = augmented_image[0].numpy() / 255.0

    #add the image to the plot, to the current i position
    ax = fig.add_subplot(3, 3, i,xticks=[],yticks=[])
    ax.imshow(augmented_showable)
    ax.set_title("Augmented " + str(i-1))

  #and we are done
  plt.show()

# Download the data

We use the [requests module](https://requests.readthedocs.io/en/master/) to download the image from a web url to a local directory.

In [ ]:
import requests

#let's create a response object...
response = requests.get(IMAGE_URL, stream=True)

#...and do a little check on the available meta-data
print(response.status_code)
print(response.headers['content-type'])
print(response.url)

If all went well we should see a 200 status code (= OK). If you see something different (especially codes starting with a 4, such as 404) something bad may have happened and you may want to check the list of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).

It's now time to write the data to local storage.

In [ ]:
with open(IMG_FILENAME, 'wb') as f:
  #the actual data payload is accessed via .content field
  f.write(response.content)

# Data augmentations

## Flips

We instantiate a minimal `Sequential` model with only one [RandomFlip](https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomFlip) layer that does random horizontal flips.



In [ ]:
# Define a minimal augmentation model with random horizontal flips
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal")
])

generate_plot_pics(data_augmentation, IMG_FILENAME)

## Other augmentations

Apart from `RandomFlip`, several other types of image data augmentation are available:

* [RandomContrast](https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomContrast)
* [RandomCrop](https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomCrop)
* [RandomRotation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomRotation)
* [RandomTranslation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomTranslation)
* [RandomZoom](https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomZoom)

Some require extra parameters, and can generate quite different results. For example when applying a translation, what to do with the newly generated space in the image? The following code creates a rabdin vertical shift up to 50% of the image, but applies two different filling strategies:

In [ ]:
# vertical shifts, up to 50%, filling with the nearest line of pixels
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomTranslation(height_factor=0.5, width_factor=0, fill_mode="nearest")
])

generate_plot_pics(data_augmentation, IMG_FILENAME)

In [ ]:
# vertical shifts, up to 50%, filling with the mirrored image
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomTranslation(height_factor=0.5, width_factor=0, fill_mode="reflect")
])

generate_plot_pics(data_augmentation, IMG_FILENAME)

# Exercise: exploring data augmentation

* update the `Randomflip` code so that it does vertical flips. Or vertical *and* horizontal
  * **EXTRA** can you find what is the chance of flips? Hint: you may need to look at the source code on github...
* pick one (or more!) data augmentation that we have not tried so far and see the results
* implement a multi-augmentation model that does at least three image modifications

In [ ]:
######## YOUR CODE HERE ########
################################